In [133]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from finta import TA
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import jaccard_score
import seaborn as sn
from tabulate import tabulate
from xgboost import XGBClassifier
from ta import add_all_ta_features
from sklearn.feature_selection import RFE

In [134]:
WINDOW = 8 # number of rows to look ahead to see what the price did
FETCH_INTERVAL = "60m"  # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
INTERVAL = '1y'     # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
symbol = 'AAPL'      # Symbol of the desired stock
ROWS_TO_PREDICT = 128
# one day 16 rows of data

In [135]:
data = pd.read_csv('C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\preprocess_new\\GOOG_1y_17_diff_11_05_2021 21_44_49_full.csv', index_col = 0)
data






,open,high,low,close,Adj Close,volume,open_1,open_2,open_3,open_4,...,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,close_shift,class_column
Datetime,,,,,,,,,,,,,,,,,,,,,
2020-05-12 13:30:00-04:00,1394.750000,1399.160034,1392.589966,1396.579956,1396.579956,101864,-8.810059,-9.319946,-3.074951,-12.369995,...,1396.579956,0.000000,0.000000,0.000000,0.000000,-21.866920,0.000000,0.000000,1351.0000,-1
2020-05-12 14:30:00-04:00,1396.579956,1397.800049,1388.449951,1389.969971,1389.969971,119507,1.829956,-6.980103,-7.489990,-1.244995,...,1393.720436,0.000000,1.364163,0.272833,1.091330,-0.473298,-0.474422,-0.473298,1352.3439,-1
2020-05-12 15:30:00-04:00,1390.805786,1392.000000,1374.770020,1374.770020,1374.770020,223831,-5.774170,-3.944214,-12.754272,-13.264160,...,1385.410775,0.000000,9.647961,2.147858,7.500102,-1.093545,-1.099568,-1.561668,1350.0000,-1
2020-05-12 16:00:00-04:00,1375.740000,1375.740000,1373.000000,1373.000000,1373.000000,0,-15.065786,-20.839956,-19.010000,-27.820059,...,1379.790950,0.000000,0.201367,1.758560,-1.557193,-0.128750,-0.128833,-1.688407,1347.5200,-1
2020-05-12 17:00:00-04:00,1369.500000,1369.500000,1369.500000,1369.500000,1369.500000,0,-6.240000,-21.305786,-27.079956,-25.250000,...,1375.171529,0.000000,-8.431366,-0.279425,-8.151941,-0.254916,-0.255242,-1.939019,1335.9600,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-11 12:30:00-04:00,2311.649902,2319.840088,2302.368896,2318.750000,2318.750000,100865,-4.765137,12.649902,19.789795,11.659902,...,2306.335849,-0.713365,30.067301,8.438716,21.628585,0.352283,0.351664,66.030594,NaN,0
2021-05-11 13:30:00-04:00,2318.475098,2322.000000,2308.100098,2320.695068,2320.695068,99874,6.825195,2.060059,19.475098,26.614990,...,2306.632664,-0.731668,25.661071,11.883187,13.777884,0.083884,0.083849,66.169868,NaN,0
2021-05-11 14:30:00-04:00,2320.280029,2321.979980,2305.729980,2309.737793,2309.737793,131604,1.804932,8.630127,3.864990,21.280029,...,2306.658155,-1.122122,23.956456,14.297841,9.658615,-0.472155,-0.473273,65.385289,NaN,0


In [136]:

# Class divide
data['class_column'].value_counts()

 1    1138
 0    1125
-1    1121
Name: class_column, dtype: int64

In [137]:
# del (data['close'])
del (data['close_shift'])
data = data.dropna()
train_set = data.iloc[:-ROWS_TO_PREDICT]
train_set = train_set.iloc[:-WINDOW] # optional drop last n rows (avoid of data leak)
test_set =data.iloc[-ROWS_TO_PREDICT:]

In [138]:
data['class_column'].value_counts()

 1    1138
 0    1125
-1    1120
Name: class_column, dtype: int64

In [139]:
train_set

,open,high,low,close,Adj Close,volume,open_1,open_2,open_3,open_4,...,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,class_column
Datetime,,,,,,,,,,,,,,,,,,,,,
2020-05-12 14:30:00-04:00,1396.579956,1397.800049,1388.449951,1389.969971,1389.969971,119507,1.829956,-6.980103,-7.489990,-1.244995,...,0.000000,1393.720436,0.000000,1.364163,0.272833,1.091330,-0.473298,-0.474422,-0.473298,-1
2020-05-12 15:30:00-04:00,1390.805786,1392.000000,1374.770020,1374.770020,1374.770020,223831,-5.774170,-3.944214,-12.754272,-13.264160,...,0.000000,1385.410775,0.000000,9.647961,2.147858,7.500102,-1.093545,-1.099568,-1.561668,-1
2020-05-12 16:00:00-04:00,1375.740000,1375.740000,1373.000000,1373.000000,1373.000000,0,-15.065786,-20.839956,-19.010000,-27.820059,...,0.000000,1379.790950,0.000000,0.201367,1.758560,-1.557193,-0.128750,-0.128833,-1.688407,-1
2020-05-12 17:00:00-04:00,1369.500000,1369.500000,1369.500000,1369.500000,1369.500000,0,-6.240000,-21.305786,-27.079956,-25.250000,...,0.000000,1375.171529,0.000000,-8.431366,-0.279425,-8.151941,-0.254916,-0.255242,-1.939019,-1
2020-05-12 19:00:00-04:00,1371.850000,1371.850000,1369.000000,1369.000000,1369.000000,0,2.350000,-3.890000,-18.955786,-24.729956,...,-2.952333,1372.357803,0.000000,-16.320356,-3.487611,-12.832745,-0.036510,-0.036516,-1.974821,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-29 12:30:00-04:00,2409.669922,2423.510010,2408.229980,2423.510010,2423.510010,126627,-19.410156,-8.709961,-0.660156,18.569922,...,16.422610,2408.416692,1.677771,13.845723,0.257032,13.588691,0.600235,0.598441,73.531777,-1
2021-04-29 13:30:00-04:00,2423.449951,2432.750000,2420.239990,2429.209961,2429.209961,165803,13.780029,-5.630127,5.070068,13.119873,...,20.125261,2409.921594,1.842992,13.120638,2.829753,10.290885,0.235194,0.234918,73.939913,-1
2021-04-29 14:30:00-04:00,2429.770020,2436.520020,2427.495850,2428.340088,2428.340088,189545,6.320068,20.100098,0.689941,11.390137,...,20.298022,2411.487422,1.878279,13.559216,4.975646,8.583570,-0.035809,-0.035815,73.877627,-1


In [140]:
y = data['class_column']
features = [x for x in data.columns if x not in ['class_column']]
x = data[features]
scaler = MinMaxScaler()
# x = pd.DataFrame(scaler.fit_transform(x.values), columns=x.columns, index=x.index)
x_train= x.iloc[:-ROWS_TO_PREDICT]
y_train= y.iloc[:-ROWS_TO_PREDICT]
x_test =x.iloc[-ROWS_TO_PREDICT:]
y_test=y.iloc[-ROWS_TO_PREDICT:]

In [141]:
classifiers = dict()

classifiers['DecisionTreeClassifier 1'] = DecisionTreeClassifier(max_depth=10, random_state=0,criterion='gini',splitter='best')
classifiers['DecisionTreeClassifier 2'] = DecisionTreeClassifier(max_depth=10, random_state=0,criterion='gini',splitter='random')
classifiers['DecisionTreeClassifier 3'] = DecisionTreeClassifier(max_depth=10, random_state=0,criterion='entropy',splitter='best')
classifiers['RandomForestClassifier 4'] = RandomForestClassifier(n_estimators=1000, max_depth=3, random_state=0,criterion='gini', n_jobs = -1)
classifiers['RandomForestClassifier 5'] = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0,criterion='entropy', n_jobs = -1)
classifiers['GradientBoostingClassifier 1'] = GradientBoostingClassifier(n_estimators=100,random_state=0,criterion='friedman_mse',max_depth=3, learning_rate=0.1)
classifiers['GradientBoostingClassifier 2'] = GradientBoostingClassifier(n_estimators=1000,random_state=0,criterion='friedman_mse',max_depth=3, learning_rate=0.1)

In [142]:
def print_conf_matrix(test_y, predict, name):
    matrix = confusion_matrix(test_y, predict,  labels=[-1, 0, 1])
    print(matrix)
    ax= plt.subplot()
    sn.heatmap(matrix, annot=True, ax = ax) #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted labels', color='white')
    ax.set_ylabel('True labels', color='white')
    ax.set_title(f'Confusion Matrix for {name}' , color='white')
    ax.xaxis.set_ticklabels(['-1','0', '1'], color='white')
    ax.yaxis.set_ticklabels(['-1','0', '1'], color='white')
    plt.show()

def train_model(model,train_x, train_y):
    model.fit(train_x, train_y)

In [143]:
predictions_train= dict()
score_train = dict()
predictions= dict()
score = dict()
for k,v in classifiers.items():
    print("Calculate: ", k)
    train_model(v,x_train,y_train)
    predictions_train[k] = v.predict(x_train)
    score_train[k] = accuracy_score(y_train.values, predictions_train[k])
    predictions[k] = v.predict(x_test)
    score[k] = accuracy_score(y_test.values, predictions[k])
    print('Score_train: ',  score_train[k] )
    print('Score: ',  score[k] )
    # print_conf_matrix(test_y, predictions[k], k)

headers = ["Classifier type", "Accuracy"]
score_df = pd.DataFrame(score.items(), columns=headers)
print(tabulate(score_df, headers, tablefmt="psql"))


Calculate:  DecisionTreeClassifier 1
Score_train:  0.8365591397849462
Score:  0.2734375
Calculate:  DecisionTreeClassifier 2
Score_train:  0.7523809523809524
Score:  0.2578125
Calculate:  DecisionTreeClassifier 3
Score_train:  0.8258064516129032
Score:  0.34375
Calculate:  RandomForestClassifier 4
Score_train:  0.5244239631336406
Score:  0.3046875
Calculate:  RandomForestClassifier 5
Score_train:  0.4706605222734255
Score:  0.390625
Calculate:  GradientBoostingClassifier 1
Score_train:  0.8844854070660523
Score:  0.3046875
Calculate:  GradientBoostingClassifier 2
Score_train:  1.0
Score:  0.296875
+----+------------------------------+------------+
|    | Classifier type              |   Accuracy |
|----+------------------------------+------------|
|  0 | DecisionTreeClassifier 1     |   0.273438 |
|  1 | DecisionTreeClassifier 2     |   0.257812 |
|  2 | DecisionTreeClassifier 3     |   0.34375  |
|  3 | RandomForestClassifier 4     |   0.304688 |
|  4 | RandomForestClassifier 5     | 

In [144]:
filename_to_export = f'../data/results/{symbol}_{WINDOW}_{ROWS_TO_PREDICT}_{ datetime.now().strftime("%d_%m_%Y %H_%M_%S")}.csv'
score_df.to_csv(filename_to_export, index=False)

In [145]:
rfe = RFE(classifiers['RandomForestClassifier 5'],10)
fited = rfe.fit(x_train, y_train)
rfe

C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


RFE(estimator=RandomForestClassifier(criterion='entropy', max_depth=2,
                                     n_jobs=-1, random_state=0),
    n_features_to_select=10)

In [146]:
names = x.columns
columns=[]
for i in range(len(fited.support_)):
    if fited.support_[i]:
        columns.append(names[i])

print("Columns with predictive power:", columns )
columns = columns + ['high', 'low', 'volume', 'open']
columns

Columns with predictive power: ['volume_adi', 'volume_cmf', 'volume_sma_em', 'volatility_bbm', 'volatility_bbp', 'volatility_dcm', 'trend_sma_slow', 'trend_psar_down', 'momentum_stoch_signal', 'momentum_kama']


['volume_adi',
 'volume_cmf',
 'volume_sma_em',
 'volatility_bbm',
 'volatility_bbp',
 'volatility_dcm',
 'trend_sma_slow',
 'trend_psar_down',
 'momentum_stoch_signal',
 'momentum_kama',
 'high',
 'low',
 'volume',
 'open']

In [147]:
x_test_cropped = x_test[columns]
x_train_cropped = x_train[columns]
x_train_cropped

,volume_adi,volume_cmf,volume_sma_em,volatility_bbm,volatility_bbp,volatility_dcm,trend_sma_slow,trend_psar_down,momentum_stoch_signal,momentum_kama,high,low,volume,open
Datetime,,,,,,,,,,,,,,
2020-05-12 14:30:00-04:00,-5.879152e+04,-0.265579,-21515.700800,1393.274963,0.250000,1393.804993,1393.274963,-1.000000,37.461115,1393.720436,1397.800049,1388.449951,119507,1396.579956
2020-05-12 15:30:00-04:00,-2.826225e+05,-0.634819,-48245.919859,1387.106649,0.162238,1386.965027,1387.106649,1399.160034,24.974076,1385.410775,1392.000000,1374.770020,223831,1390.805786
2020-05-12 16:00:00-04:00,-2.826225e+05,-0.634819,-48245.919859,1383.579987,0.235296,1386.080017,1383.579987,1398.672234,4.730805,1379.790950,1375.740000,1373.000000,0,1375.740000
2020-05-12 17:00:00-04:00,-2.826225e+05,-0.634819,-48245.919859,1380.763989,0.233432,1384.330017,1380.763989,1397.645345,0.000000,1375.171529,1369.500000,1369.500000,0,1369.500000
2020-05-12 19:00:00-04:00,-2.826225e+05,-0.634819,-48245.919859,1378.803324,0.268643,1384.080017,1378.803324,1395.956624,0.000000,1372.357803,1371.850000,1369.000000,0,1371.850000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-30 04:00:00-04:00,1.462892e+07,0.157081,25149.904115,2409.459749,0.524487,2094.615010,2406.005577,2436.520020,96.993314,2414.573339,2419.680000,2410.970000,0,2419.680000
2021-04-30 05:00:00-04:00,1.462892e+07,0.157081,25149.904115,2411.026749,0.594184,2094.615010,2406.141729,2436.520020,96.735445,2414.621098,2416.990000,2412.030000,0,2413.020000
2021-04-30 06:00:00-04:00,1.462892e+07,0.157081,25149.904115,2412.445749,0.490721,2094.615010,2406.293652,2436.520020,96.586768,2414.474366,2415.860000,2409.450000,0,2415.860000


In [148]:
classifiers_boosted = dict()
classifiers_boosted['GradientBoostingClassifier 1'] = GradientBoostingClassifier(n_estimators=100,random_state=0,criterion='friedman_mse',max_depth=3, learning_rate=0.1)
classifiers_boosted['GradientBoostingClassifier 2'] = GradientBoostingClassifier(n_estimators=1000,random_state=0,criterion='friedman_mse',max_depth=3, learning_rate=0.3)
classifiers_boosted['GradientBoostingClassifier 3'] = GradientBoostingClassifier(n_estimators=1000,random_state=0,criterion='friedman_mse',max_depth=2, learning_rate=0.5)
classifiers_boosted['GradientBoostingClassifier 4'] = GradientBoostingClassifier(n_estimators=1000,random_state=0,criterion='friedman_mse',max_depth=2, learning_rate=0.8)

In [149]:
predictions_train= dict()
score_train = dict()
predictions= dict()
score = dict()
for k,v in classifiers_boosted.items():
    print("Calculate: ", k)
    train_model(v,x_train_cropped,y_train)
    predictions_train[k] = v.predict(x_train_cropped)
    score_train[k] = accuracy_score(y_train.values, predictions_train[k])
    predictions[k] = v.predict(x_test_cropped)
    score[k] = accuracy_score(y_test.values, predictions[k])
    print('Score train: ',  score_train[k] )
    print('Score: ',  score[k] )
    # print_conf_matrix(test_y, predictions[k], k)


Calculate:  GradientBoostingClassifier 1
Score train:  0.8003072196620584
Score:  0.296875
Calculate:  GradientBoostingClassifier 2
Score train:  1.0
Score:  0.3125
Calculate:  GradientBoostingClassifier 3
Score train:  1.0
Score:  0.2890625
Calculate:  GradientBoostingClassifier 4
Score train:  1.0
Score:  0.265625


In [150]:
import xgboost as xgb
model = xgb.XGBClassifier(nthread =-1,max_depth=14,n_estimators=1000,
                          eta =0.2)
model.fit(x_train,y_train)
predicted_train = model.predict(x_train)
predicted_test = model.predict(x_test)
print("------------")
print(f'max_depth: {14}')
score_train['XGBClassifier'] = accuracy_score(y_train.values, predicted_train)
score['XGBClassifier'] = accuracy_score(y_test.values, predicted_test)
print(score_train['XGBClassifier'])
print(score['XGBClassifier'])
print("------------")

[21:51:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


------------
max_depth: 14
1.0
0.296875
------------


In [151]:
model = xgb.sklearn.XGBRFClassifier(n_jobs=-1,max_depth=12,n_estimators =100,eta=0.4)
model.fit(x_train,y_train)
predicted_train = model.predict(x_train)
predicted_test = model.predict(x_test)
print("------------")
print(f'eta: ')
score_train['XGBRFClassifier'] = accuracy_score(y_train.values, predicted_train)
score['XGBRFClassifier'] = accuracy_score(y_test.values, predicted_test)
print(accuracy_score(y_train.values, predicted_train))
print(accuracy_score(y_test.values, predicted_test))
print("------------")

[21:51:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
------------
eta: 
0.9920122887864823
0.265625
------------


In [152]:
headers = ["Classifier type", "Accuracy"]
score_df = pd.DataFrame(score.items(), columns=headers)
print(tabulate(score_df, headers, tablefmt="psql"))
score_df.to_csv(filename_to_export,mode='a', index=False, header=False)

+----+------------------------------+------------+
|    | Classifier type              |   Accuracy |
|----+------------------------------+------------|
|  0 | GradientBoostingClassifier 1 |   0.296875 |
|  1 | GradientBoostingClassifier 2 |   0.3125   |
|  2 | GradientBoostingClassifier 3 |   0.289062 |
|  3 | GradientBoostingClassifier 4 |   0.265625 |
|  4 | XGBClassifier                |   0.296875 |
|  5 | XGBRFClassifier              |   0.265625 |
+----+------------------------------+------------+
